<a href="https://colab.research.google.com/github/protovici/PyTorch_Lightning/blob/main/lightning_cnn_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>